In [2]:
!pip3 install adafruit-circuitpython-servokit

   ---------------------------------------- 0.0/740.9 kB ? eta -:--:--
   ---------------------------------------- 740.9/740.9 kB 7.3 MB/s  0:00:00

   ----------------------------------------  0/16 [pyserial]
   ----------------------------------------  0/16 [pyserial]
   ----------------------------------------  0/16 [pyserial]
   ----------------------------------------  0/16 [pyserial]
   ----------------------------------------  0/16 [pyserial]
   ----------------------------------------  0/16 [pyserial]
   ----------------------------------------  0/16 [pyserial]
   ----------------------------------------  0/16 [pyserial]
   ----------------------------------------  0/16 [pyserial]
   ----------------------------------------  0/16 [pyserial]
   ----------------------------------------  0/16 [pyserial]
   -- -------------------------------------  1/16 [Adafruit-PlatformDetect]
   -- -------------------------------------  1/16 [Adafruit-PlatformDetect]
   ----- -------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
#Class Structure for robot arm

import time
from adafruit_servokit import ServoKit

class robo_arm:
    
    
    def __init__(self):
        self.kit = ServoKit(channels=16)
        print(self.kit._items)
        print(self.kit._pca)
        print

        #set default angles
        self.servo_angles = {
            "base" : 90,
            "shoulder" : 0,
            "elbow" : 135,
            "wrist" : 90,
            "hand" : 180
        }

        self.servo_channel = {
        "base" : 0,
        "shoulder" : 12,
        "elbow" : 15,
        "wrist" : 14,
        "hand" : 13
        }

        for servo in self.servo_angles:
            try:
                ch = self.servo_channel[servo]
                # print(ch)
                self.kit.servo[ch].angle = self.servo_angles[servo] 
            except Exception:
                print("failed to update channel: "+ ch)
                pass

        print("ROBO-ARM INIT COMPLETE")    
    # END OF INIT
     
    # # define new servo angle -- by keyword atm 
    # def set_angle(self, **servos):    
        
    #     for servo, angle in servos.items():
    
    #         try:
    #             ch = self.servo_channel[servo]
    #             print(ch)
    #             #Save the angle in object dict
    #             self.servo_angles[servo] = angle
    #             #Update that servo to be the angle
    #             kit.servo[ch].angle = self.servo_angles[angle]
    #         except Exception:
    #             pass
    
    # simple update for the angles we changed
    def update(self, delay = 1):
        #optional delay
        time.sleep(delay)

        #iterate through servos and update the channel
        for servo in self.servo_angles:
            try:
                ch = self.servo_channel[servo]
                print("update channgel: " + ch)
                self.kit.servo[ch].angle = self.servo_angles[servo]  
            except Exception:
                pass
        print("Robo-Arm Angle Update Complete")

    # def get_angles(self):

        # return self.servo_angles
        #easy pose /update -- just get the order right
    def pose(self, shoulder = 90, elbow = 180, wrist = 45):
        
        #shoulder is 7V and not used atm
        self.servo_angles["shoulder"] = shoulder
        self.servo_angles["elbow"] = elbow
        self.servo_angles["wrist"] = wrist
        # self.servo_angles["hand"] = hand
        self.update()

    #return to netrual -- update included
    def pose_neutral(self):
        self.servo_angles["shoulder"] = 180
        self.servo_angles["elbow"] = 135
        self.servo_angles["wrist"] = 45
        # self.servo_angles["hand"]  = 180
        self.update()

    def pose_ramp(self,shoulder, elbow, wrist):
        step_deg = 1.0
        step_sec = 0.005
        #determine the direction of travel for each limb

        # negative if set angle is less than current angle
        # shoulder_dir = self.servo_angles["shoulder"] - shoulder
        elbow_diff = elbow - self.servo_angles["elbow"]  
        wrist_diff = wrist - self.servo_angles["wrist"]
        
        while(
            # self.servo_angles["shoulder"] != shoulder + step
            # self.servo_angles["elbow"] < elbow
            abs(elbow_diff) - step_deg > 0 or abs(wrist_diff) - step_deg > 0
        ):
            if(elbow_diff > 0):
                self.servo_angles["elbow"] = self.servo_angles["elbow"] + step_deg
            elif(elbow_diff < 0):
                self.servo_angles["elbow"] = self.servo_angles["elbow"] - step_deg

            if(wrist_diff > 0):
                self.servo_angles["wrist"] = self.servo_angles["wrist"] + step_deg
            elif(wrist_diff < 0):
                self.servo_angles["wrist"] = self.servo_angles["wrist"] - step_deg
            
            self.update(0)
            elbow_diff = elbow - self.servo_angles["elbow"]
            wrist_diff = wrist - self.servo_angles["wrist"]
            time.sleep(step_sec)



    #Set the grib strength, barely touching is default. 180 is wide open
    def grab(self, strength = 120):
        self.servo_angles["hand"] = strength
        self.update(0.5)
        time.sleep(0.5)

    #release grip to 100% open
    def release(self):
        self.servo_angles["hand"] = 180
        self.update()
    

ModuleNotFoundError: No module named 'pkg_resources'

In [ ]:
#Simple test for Arm
arm = robo_arm()

print("Robo-Arm's Servo Channels used on breakout board")
print(arm.servo_channel)

print("Robo-Arm's Initial servo angles ")
print(arm.servo_angles)


#Simple pose -- snap right to angles
#angles are shoulder, elbow, wrist -- use grab to change hand angle
arm.pose(0, 0, 180)

arm.grab()

arm.pose_ramp(0,45,120)

arm.release()
